In [ ]:
from datetime import datetime
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)

In [ ]:
import os.path
import sys
motornet_home = os.path.expanduser("~") + "/github/MotorNet"
sys.path.append(motornet_home)

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import motornet as mn
from motornet.plants.muscles import RigidTendonHillMuscleThelen

# Disable all GPUS
tf.config.set_visible_devices([], 'GPU')
visible_devices = tf.config.get_visible_devices()
for device in visible_devices:
    assert device.device_type != 'GPU'

%load_ext autoreload
%autoreload 2

In [ ]:
import plg_tasks

In [ ]:
# SET UP FFs
b = 8
FF_NF  = tf.convert_to_tensor(np.array([[0, 0],[ 0,0]]), dtype=tf.float32)
FF_CCW = tf.convert_to_tensor(np.array([[0,-b],[ b,0]]), dtype=tf.float32)
FF_CW  = tf.convert_to_tensor(np.array([[0, b],[-b,0]]), dtype=tf.float32)

In [ ]:
def create_network():
    # SET UP PLANT
    plant = mn.plants.RigidTendonArm26(muscle_type=RigidTendonHillMuscleThelen(), visual_delay=0.07, proprioceptive_delay=0.02)
    # SET UP NETWORK
    n_units = 64
    network = mn.nets.layers.GRUNetwork(plant=plant, n_units=n_units, kernel_regularizer=1e-7, name='network', recurrent_regularizer=1e-5)
    # SET UP TASK
    start_position = [0.785, 1.570] # [45,90] deg
    go_cue_range = (.100, .300)
    task = plg_tasks.CentreOutFF(network=network, start_position=start_position, go_cue_range=go_cue_range)
    task.network.do_recompute_inputs = True
    task.network.recompute_inputs = task.recompute_inputs
    task._loss_weights["cartesian position"] = [1.0]
    task._loss_weights["muscle state"] = [5.0]
    task._loss_weights["gru_hidden_0"] = [0.1]
    # SET UP RNN
    rnn = tf.keras.layers.RNN(cell=network, return_sequences=True, name='RNN')
    input_dict = task.get_input_dict_layers()
    state0 = task.get_initial_state_layers()
    states_out = rnn(input_dict, initial_state=state0)
    # SET UP MAIN NETWORK OBJECT
    nn = mn.nets.MotorNetModel(inputs=[input_dict, state0], outputs=states_out, name='model', task=task)
    nn.compile(optimizer=tf.optimizers.Adam(clipnorm=1.), loss=task.losses, loss_weights=task.loss_weights)
    return nn

In [ ]:
nn = create_network()
nn.summary()

In [ ]:
nn.task.print_losses()

# TRAIN NETWORK TO REACH IN A NF

In [ ]:
# TRAIN NETWORK on point to point reaches between random targets in the arm's workspace
# IN A NULL FIELD

FF = FF_NF
condition = "train"

n_t = 100
n_batches = 1024
batch_size = 64
# this callback logs training information for each batch passed, rather than for each epoch.
callbacks = [mn.nets.callbacks.BatchLogger(), mn.nets.callbacks.TrainingPlotter(nn.task, plot_freq=256, plot_trials=8, plot_n_t=n_t)]
for i in range(3):
    [inputs, targets, init_states] = nn.task.generate(n_timesteps=n_t, batch_size=n_batches*batch_size, condition=condition)
    nn.task.FF_matvel = FF
    h = nn.fit(x=[inputs, init_states], y=targets, verbose=1, epochs=1, batch_size=batch_size, shuffle=False, callbacks=callbacks)

In [ ]:
# save model weights
weight_file = "save" + os.path.sep + "example_centeroutFF" + "_" + "weights"
nn.save_weights(weight_file)

# LOAD TRAINED NETWORK

In [ ]:
weight_file = "save" + os.path.sep + "example_centeroutFF" + "_" + "weights"
nn = create_network()
nn.load_weights(weight_file).expect_partial()

In [ ]:
def hand_to_joints(xy, L1, L2):
      j = np.zeros(np.shape(xy))
      j[:,:,1] = np.arccos((xy[:,:,0]**2 + xy[:,:,1]**2 - L1**2 - L2**2) / (2*L1*L2))
      j[:,:,0] = np.arctan2(xy[:,:,1],xy[:,:,0]) - np.arctan2(L2*np.sin(j[:,:,1]),L1+(L2*np.cos(j[:,:,1])))
      return j

def plot_simulations(xy, target_xy):
  target_x = target_xy[:, -1, 0]
  target_y = target_xy[:, -1, 1]
  plt.figure()
  nmov,ntime,_ = np.shape(xy)
  for i in range(nmov):
        plt.plot(xy[i,:,0],xy[i,:,1],color="b",linewidth=1.0)
  plt.scatter(target_x, target_y)
  plt.axis("equal")
  plt.xlabel("X (m)")
  plt.ylabel("Y (m)")
  plt.show()

def plot1trial(inputs, results, targets_j, trial=0):
    j_results = results['joint position']
    m_results = results['muscle state']

    plt.figure(figsize=(14, 2.5)).set_tight_layout(True)

    plt.subplot(141)
    plt.plot(j_results[trial, :, 0]*180/np.pi, label='sho')
    plt.plot(j_results[trial, :, 1]*180/np.pi, label='elb')
    plt.plot(targets_j[trial, :, 0]*180/np.pi, '--')
    plt.plot(targets_j[trial, :, 1]*180/np.pi, '--')
    plt.axvline(np.where(inputs["inputs"][trial, :, -1] != 1)[0][0] - nn.task.network.visual_delay, c='grey')
    plt.legend()
    plt.xlabel('time (ms)')
    plt.ylabel('angle (deg)')

    plt.subplot(142)
    plt.plot(j_results[trial, :, 2], label='sho')
    plt.plot(j_results[trial, :, 3], label='elb')
    plt.legend()
    plt.xlabel('time (ms)')
    plt.ylabel('angle velocity (rad/sec)')

    plt.subplot(143)
    plt.plot(m_results[trial, :, 0, :])
    plt.xlabel('time (ms)')
    plt.ylabel('activation (a.u.)')
    plt.legend(["PEC","DEL","BRA","TRI","BIC","TR2"], loc=1)

    plt.subplot(144)
    plt.plot(m_results[trial, :, -1, :])
    plt.xlabel('time (ms)')
    plt.ylabel('force (N)')
    plt.legend(["PEC","DEL","BRA","TRI","BIC","TR2"], loc=1)

    plt.show()



In [ ]:
# TEST THE NETWORK ON THE CENTRE-OUT REACHING TASK
# IN A NULL FIELD

FF = FF_NF
condition = "test"

n_mov_circle = 8 # number of movement directions around the unit circle

n_t = 100
nn.task.angular_step = 360 / n_mov_circle
[inputs, targets, init_states] = nn.task.generate(n_timesteps=n_t, batch_size=n_mov_circle, condition=condition)
nn.task.FF_matvel = FF
results = nn([inputs, init_states], training=False)

plot_simulations(xy=results["cartesian position"], target_xy=targets)

L1,L2 = nn.task.network.plant.skeleton.L1, nn.task.network.plant.skeleton.L2
targets_j = hand_to_joints(targets, L1, L2)
n_mov = np.shape(results["joint position"])[0]
for i in range(n_mov):
    plot1trial(inputs, results, targets_j, i)

In [ ]:
# TEST THE NETWORK ON THE CENTRE-OUT REACHING TASK
# IN A CCW CURL FIELD

FF = FF_CCW
condition = "test"

n_mov_circle = 8

n_t = 150
nn.task.angular_step = 360 / n_mov_circle
[inputs, targets, init_states] = nn.task.generate(n_timesteps=n_t, batch_size=n_mov_circle, condition=condition)
nn.task.FF_matvel = FF
results = nn([inputs, init_states], training=False)

plot_simulations(xy=results["cartesian position"], target_xy=targets)

L1,L2 = nn.task.network.plant.skeleton.L1, nn.task.network.plant.skeleton.L2
targets_j = hand_to_joints(targets, L1, L2)
n_mov = np.shape(results["joint position"])[0]
for i in range(n_mov):
    plot1trial(inputs, results, targets_j, i)

In [ ]:
# TRAIN NETWORK on centre-out reaches
# IN A CCW FF

FF = FF_CCW
condition = "adapt"

n_t = 100
n_batches = 256
batch_size = 64
# this callback logs training information for each batch passed, rather than for each epoch.
callbacks = [mn.nets.callbacks.BatchLogger()]
for i in range(1):
    [inputs, targets, init_states] = nn.task.generate(n_timesteps=n_t, batch_size=batch_size*n_batches, condition=condition)
    nn.task.FF_matvel = FF
    h = nn.fit(x=[inputs, init_states], y=targets, verbose=1, epochs=1, batch_size=batch_size, shuffle=False, callbacks=callbacks)

In [ ]:
# TEST THE NETWORK ON THE CENTRE-OUT REACHING TASK
# IN A CCW CURL FIELD

FF = FF_CCW
condition = "test"

n_mov_circle = 8

n_t = 100
nn.task.angular_step = 360 / n_mov_circle
[inputs, targets, init_states] = nn.task.generate(n_timesteps=n_t, batch_size=n_mov_circle, condition=condition)
nn.task.FF_matvel = FF
results = nn([inputs, init_states], training=False)

plot_simulations(xy=results["cartesian position"], target_xy=targets)

L1,L2 = nn.task.network.plant.skeleton.L1, nn.task.network.plant.skeleton.L2
targets_j = hand_to_joints(targets, L1, L2)
n_mov = np.shape(results["joint position"])[0]
for i in range(n_mov):
    plot1trial(inputs, results, targets_j, i)

In [ ]:
# TEST THE NETWORK ON THE CENTRE-OUT REACHING TASK
# IN A NULL FIELD

FF = FF_NF
condition = "test"

n_mov_circle = 8

n_t = 120
nn.task.angular_step = 360 / n_mov_circle
[inputs, targets, init_states] = nn.task.generate(n_timesteps=n_t, batch_size=n_mov_circle, condition=condition)
nn.task.FF_matvel = FF
results = nn([inputs, init_states], training=False)

plot_simulations(xy=results["cartesian position"], target_xy=targets)

L1,L2 = nn.task.network.plant.skeleton.L1, nn.task.network.plant.skeleton.L2
targets_j = hand_to_joints(targets, L1, L2)
n_mov = np.shape(results["joint position"])[0]
for i in range(n_mov):
    plot1trial(inputs, results, targets_j, i)

In [ ]:
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)